In [2]:
import requests
import numpy as np
import pandas as pd
import time
from datetime import datetime
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler


KeyboardInterrupt: 

In [ ]:
def send_telegram_message(message):
    token = '7815331489:AAEe0BYA1lKdsFP88-4ta1d9Szg95bQYXw0'
    chat_id = '1035839883'
    url = f"https://api.telegram.org/bot{token}/sendMessage"
    payload = {'chat_id': chat_id, 'text': message}
    requests.post(url, data=payload)


In [ ]:
def fetch_hourly_gold_data():
    url = "https://api.twelvedata.com/time_series?symbol=XAU/USD&interval=1h&outputsize=48&apikey=your-twelvedata-api-key"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data["values"])
        df = df[::-1]  # reverse to chronological order
        df["datetime"] = pd.to_datetime(df["datetime"])
        df["price"] = df["close"].astype(float)
        return df[["datetime", "price"]].reset_index(drop=True)
    else:
        return None


In [ ]:
def prepare_data(data, window_size=12):
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(data[["price"]])
    
    X, y = [], []
    for i in range(window_size, len(scaled)):
        X.append(scaled[i-window_size:i])
        y.append(scaled[i])
    
    X, y = np.array(X), np.array(y)
    return X, y, scaler


In [ ]:
def build_and_train_lstm(X, y):
    model = Sequential()
    model.add(LSTM(50, return_sequences=False, input_shape=(X.shape[1], 1)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.fit(X, y, epochs=10, batch_size=8, verbose=0)
    return model


In [ ]:
def predict_next_price(data):
    X, y, scaler = prepare_data(data)
    X = X.reshape((X.shape[0], X.shape[1], 1))
    model = build_and_train_lstm(X, y)
    
    last_sequence = X[-1].reshape((1, X.shape[1], 1))
    pred_scaled = model.predict(last_sequence)[0][0]
    predicted_price = scaler.inverse_transform([[pred_scaled]])[0][0]
    return predicted_price


In [ ]:
def generate_signal(predicted_price):
    if predicted_price > 2350:
        return "SELL"
    elif predicted_price < 2300:
        return "BUY"
    else:
        return "HOLD"


In [ ]:
while True:
    now = datetime.utcnow().strftime("%Y-%m-%d %H:%M UTC")
    df = fetch_hourly_gold_data()
    
    if df is not None:
        pred_price = predict_next_price(df)
        signal = generate_signal(pred_price)
        msg = f"[{now}] Predicted Gold Price (Next Hour): ${pred_price:.2f}\nSignal: {signal}"
        send_telegram_message(msg)
        print(msg)
    else:
        send_telegram_message(f"[{now}] Failed to fetch gold price data.")
        print("Failed to fetch data.")
    
    time.sleep(14400)  # sleep for 4 hours


In [ ]:

# Install Deriv API client (if not already)
!pip install deriv-api


In [ ]:

from deriv_api import DerivAPI
import asyncio

API_TOKEN = "your_deriv_api_token_here"  # Replace with your actual token
api = DerivAPI(app_id="1089")

async def connect_deriv():
    await api.connect()
    await api.authorize(API_TOKEN)
    print("Connected to Deriv API")

async def get_balance():
    balance_response = await api.send({"balance": 1})
    return float(balance_response['balance']['balance'])

async def place_trade(contract_type, amount):
    contract = {
        "buy": 1,
        "price": amount,
        "parameters": {
            "amount": amount,
            "basis": "stake",
            "contract_type": contract_type,
            "currency": "USD",
            "duration": 5,
            "duration_unit": "t",
            "symbol": "frxXAUUSD"
        }
    }
    response = await api.send(contract)
    print("Trade placed:", response)
    return response


In [ ]:

from sklearn.metrics import mean_absolute_percentage_error, r2_score, mean_squared_error
import numpy as np

# Evaluate the prediction results
mape = mean_absolute_percentage_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"MAPE: {mape * 100:.2f}%")
print(f"R² Score: {r2:.4f}")
print(f"RMSE: {rmse:.4f}")


In [ ]:

# Trading decision logic based on prediction and threshold
async def trade_on_signal(current_price, predicted_price):
    diff = abs(predicted_price - current_price)
    if diff < 20:
        print("Price gap not significant. No trade made.")
        return

    balance = await get_balance()
    stake = round(0.2 * balance, 2)

    if predicted_price > current_price:
        print("BUY signal detected.")
        await place_trade("CALL", stake)
    else:
        print("SELL signal detected.")
        await place_trade("PUT", stake)
